In [1]:
import os

print("=== 正在扫描 Input 目录下的所有文件 ===")
# 这段代码会把 Input 文件夹里所有的文件名打印出来
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

=== 正在扫描 Input 目录下的所有文件 ===
/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg
/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv


In [2]:
import pandas as pd

# 1. 定义列名（NSL-KDD 原始 txt 没有表头，必须手动加）
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
           'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
           'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
           'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
           'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
           'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
           'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
           'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
           'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
           'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label', 'difficulty']

# 2. 读取数据 (使用你刚才扫描到的准确路径)
# 注意：我们扔掉最后一列 'difficulty'，因为它对分类没用
df_kdd = pd.read_csv('/kaggle/input/nslkdd/KDDTrain+.txt', header=None, names=columns)
df_kdd = df_kdd.drop('difficulty', axis=1)

# 3. 查看数据
print("=== NSL-KDD 数据加载成功 ===")
print(f"数据形状 (行, 列): {df_kdd.shape}")
display(df_kdd.head())

# 4. 查看标签分布（看看有多少正常，多少攻击）
print("\n=== 攻击类型分布 (前10名) ===")
print(df_kdd['label'].value_counts().head(10))

=== NSL-KDD 数据加载成功 ===
数据形状 (行, 列): (125973, 42)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal



=== 攻击类型分布 (前10名) ===
label
normal         67343
neptune        41214
satan           3633
ipsweep         3599
portsweep       2931
smurf           2646
nmap            1493
back             956
teardrop         892
warezclient      890
Name: count, dtype: int64


In [3]:
# 1. 读取 CIC-IDS (使用你扫描到的路径)
cic_path = '/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv'
print("正在读取 CIC-IDS-2017 (请稍候)...")
df_cic = pd.read_csv(cic_path)

# 2. 查看数据
print("=== CIC-IDS-2017 数据加载成功 ===")
print(f"数据形状: {df_cic.shape}")
display(df_cic.head())

# 3. 检查标签列
# 清洗版通常把标签叫 'Label'，我们确认一下分布
print("\n=== CIC-IDS 标签分布 ===")
if 'Label' in df_cic.columns:
    print(df_cic['Label'].value_counts())
else:
    print("列名列表:", df_cic.columns) # 如果没找到 Label，打印所有列名看看

正在读取 CIC-IDS-2017 (请稍候)...
=== CIC-IDS-2017 数据加载成功 ===
数据形状: (2520751, 53)


,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,...,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Max,Active Min,Idle Mean,Idle Max,Idle Min,Attack Type
0,22,1266342,41,2664,456,0,64.975610,109.864573,976,0,...,243,24,32,0.0,0,0,0.0,0,0,Normal Traffic
1,22,1319353,41,2664,456,0,64.975610,109.864573,976,0,...,243,24,32,0.0,0,0,0.0,0,0,Normal Traffic
2,22,160,1,0,0,0,0.000000,0.000000,0,0,...,243,0,32,0.0,0,0,0.0,0,0,Normal Traffic
3,22,1303488,41,2728,456,0,66.536585,110.129945,976,0,...,243,24,32,0.0,0,0,0.0,0,0,Normal Traffic
4,35396,77,1,0,0,0,0.000000,0.000000,0,0,...,290,0,32,0.0,0,0,0.0,0,0,Normal Traffic



=== CIC-IDS 标签分布 ===
列名列表: Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Length of Fwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'Average Packet Size', 'Subflow Fwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_

In [4]:
# 取出 NSL-KDD 的第一行数据
sample_row = df_kdd.iloc[0]

print("=== 核心思考：同一个数据的两种视角 ===")

# 视角 A：数值/特征视角 (给传统深度学习模型看)
# 这一串数字/符号就是 MLP/CNN 读入的东西
feature_view = [sample_row['duration'], sample_row['protocol_type'], 
                sample_row['src_bytes'], sample_row['flag']]
print(f"1. 深度学习特征输入: {feature_view}")

# 视角 B：文本/日志视角 (给 LLM 看)
# 这是一个自然语言句子，LLM 可以读懂它的语义
log_view = f"Traffic Log: Protocol is {sample_row['protocol_type']} using {sample_row['service']} service. Status flag is {sample_row['flag']}. Sent {sample_row['src_bytes']} bytes."
print(f"2. LLM 文本输入: \"{log_view}\"")

print("\n【结论】：以前做 NIDS 只用视角 A，我们接下来的计划是把视角 B 也可以利用起来！")

=== 核心思考：同一个数据的两种视角 ===
1. 深度学习特征输入: [0, 'tcp', 491, 'SF']
2. LLM 文本输入: "Traffic Log: Protocol is tcp using ftp_data service. Status flag is SF. Sent 491 bytes."

【结论】：以前做 NIDS 只用视角 A，我们接下来的计划是把视角 B 也可以利用起来！
